In [29]:
import sys
sys.path.append('../')
import pickle
from preprocessing.imdb_datareader import IMDBDataReader
from preprocessing.imdb_datareader import PopularityReader
from preprocessing.factorization_machine_transformer import  FactorizationMachineTransformer

user_item  = pickle.load(open("data/user_item.p", "rb")) 
users = pickle.load(open("data/users.p", "rb")) 
items = pickle.load(open("data/items.p", "rb")) 
train_user_item = user_item[:int(len(user_item)*0.8)]
test_user_item = user_item[int(len(user_item)*0.8):]
popreader = PopularityReader()
pop_info = popreader.read_item_data(train_user_item)
for k, v in items.items():
    if k in pop_info:
        v.append(pop_info[k])
    else:
        v.append(0)


transformer = FactorizationMachineTransformer(users, items, train_user_item)
X_train, Y_train, _, _, nFeatures = transformer.get_feature_vectors(users, items, train_user_item)
X_test, Y_test,X_cold_test, Y_cold_test, nFeatures = transformer.get_feature_vectors(users, items, test_user_item)

In [30]:
!aws s3api create-bucket --bucket recommendation-demo-yianc-0814 --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2


An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [10]:
bucket = 'recommendation-demo-yianc'
prefix = 'sagemaker/fm-movielens'
train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train')
output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)


import io,boto3
import sagemaker.amazon.common as smac


def writeDatasetToProtobuf(X, bucket, prefix, key, d_type, Y=None):
    buf = io.BytesIO()
    if d_type == "sparse":
        smac.write_spmatrix_to_sparse_tensor(buf, X, labels=Y)
    else:
        smac.write_numpy_to_dense_tensor(buf, X, labels=Y)
        
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)
 
    
train_data = writeDatasetToProtobuf(X_train, bucket, train_prefix, train_key, "sparse", Y_train)    
  
print('Output: {}'.format(output_prefix))

Output: s3://recommendation-demo-yianc/sagemaker/fm-movielens/output


In [11]:
train_data 

's3://recommendation-demo-yianc/sagemaker/fm-movielens/train/train.protobuf'

In [12]:
import sagemaker 

from sagemaker import get_execution_role
import boto3 
from sagemaker.amazon.amazon_estimator import get_image_uri
region = boto3.Session().region_name
container = get_image_uri(region, 'factorization-machines', 'latest')
container


'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


'382416733822.dkr.ecr.us-east-1.amazonaws.com/factorization-machines:latest'

In [33]:

import boto3 
import sagemaker
from sagemaker import get_execution_role

crole = get_execution_role() 

dir(sagemaker.estimator.Estimator)
fm = sagemaker.estimator.Estimator(container,
                                   crole, 
                                   train_instance_count=1, 
                                   train_instance_type='ml.c4.xlarge',
                                   output_path=output_prefix,
                                   sagemaker_session=sagemaker.Session())



fm.set_hyperparameters(
                      feature_dim=nFeatures,
                      predictor_type='binary_classifier',
                      predictor_type='regressor',
                      mini_batch_size=200,
                      num_factors=64,
                      epochs=50)

fm.fit({'train': train_data})


Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-08-16 06:19:36 Starting - Starting the training job...
2020-08-16 06:19:38 Starting - Launching requested ML instances.........
2020-08-16 06:21:21 Starting - Preparing the instances for training......
2020-08-16 06:22:24 Downloading - Downloading input data...
2020-08-16 06:22:56 Training - Downloading the training image...
2020-08-16 06:23:20 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python2.7/site-packages/pandas/util/nosetester.py:13: DeprecationWarning: Importing from numpy.testing.nosetester is deprecated, import from numpy.testing instead.
  from numpy.testing import nosetester
[08/16/2020 06:23:24 INFO 140461871777600] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'factors_lr': u'0.0001', u'linear_init_sigma': u'0.01', u'epochs': 1, u'_wd': u'1.0', u'_num_kv_servers':

[2020-08-16 06:23:35.031] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 20, "duration": 979, "num_examples": 400, "num_bytes": 8461936}
[08/16/2020 06:23:35 INFO 140461871777600] #quality_metric: host=algo-1, epoch=9, train rmse <loss>=0.475096380745
[08/16/2020 06:23:35 INFO 140461871777600] #quality_metric: host=algo-1, epoch=9, train mse <loss>=0.225716570997
[08/16/2020 06:23:35 INFO 140461871777600] #quality_metric: host=algo-1, epoch=9, train absolute_loss <loss>=0.402045952463
#metrics {"Metrics": {"update.time": {"count": 1, "max": 981.2138080596924, "sum": 981.2138080596924, "min": 981.2138080596924}}, "EndTime": 1597559015.032478, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1597559014.050807}

[08/16/2020 06:23:35 INFO 140461871777600] #progress_metric: host=algo-1, completed 20 % of epochs
#metrics {"Metrics": {"Max Batches Seen Between Resets": {"count": 1, "max": 400, 

[2020-08-16 06:23:45.036] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 40, "duration": 982, "num_examples": 400, "num_bytes": 8461936}
[08/16/2020 06:23:45 INFO 140461871777600] #quality_metric: host=algo-1, epoch=19, train rmse <loss>=0.565965683097
[08/16/2020 06:23:45 INFO 140461871777600] #quality_metric: host=algo-1, epoch=19, train mse <loss>=0.320317154443
[08/16/2020 06:23:45 INFO 140461871777600] #quality_metric: host=algo-1, epoch=19, train absolute_loss <loss>=0.462092322159
#metrics {"Metrics": {"update.time": {"count": 1, "max": 985.353946685791, "sum": 985.353946685791, "min": 985.353946685791}}, "EndTime": 1597559025.037255, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1597559024.051389}

[08/16/2020 06:23:45 INFO 140461871777600] #progress_metric: host=algo-1, completed 40 % of epochs
#metrics {"Metrics": {"Max Batches Seen Between Resets": {"count": 1, "max": 400, 

[2020-08-16 06:23:54.935] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 60, "duration": 965, "num_examples": 400, "num_bytes": 8461936}
[08/16/2020 06:23:54 INFO 140461871777600] #quality_metric: host=algo-1, epoch=29, train rmse <loss>=0.710395064173
[08/16/2020 06:23:54 INFO 140461871777600] #quality_metric: host=algo-1, epoch=29, train mse <loss>=0.504661147201
[08/16/2020 06:23:54 INFO 140461871777600] #quality_metric: host=algo-1, epoch=29, train absolute_loss <loss>=0.555065638924
#metrics {"Metrics": {"update.time": {"count": 1, "max": 967.9620265960693, "sum": 967.9620265960693, "min": 967.9620265960693}}, "EndTime": 1597559034.936159, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1597559033.967815}

[08/16/2020 06:23:54 INFO 140461871777600] #progress_metric: host=algo-1, completed 60 % of epochs
#metrics {"Metrics": {"Max Batches Seen Between Resets": {"count": 1, "max": 40

[2020-08-16 06:24:04.757] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 80, "duration": 966, "num_examples": 400, "num_bytes": 8461936}
[08/16/2020 06:24:04 INFO 140461871777600] #quality_metric: host=algo-1, epoch=39, train rmse <loss>=0.690997411295
[08/16/2020 06:24:04 INFO 140461871777600] #quality_metric: host=algo-1, epoch=39, train mse <loss>=0.477477422416
[08/16/2020 06:24:04 INFO 140461871777600] #quality_metric: host=algo-1, epoch=39, train absolute_loss <loss>=0.527522883463
#metrics {"Metrics": {"update.time": {"count": 1, "max": 968.271017074585, "sum": 968.271017074585, "min": 968.271017074585}}, "EndTime": 1597559044.757888, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1597559043.789133}

[08/16/2020 06:24:04 INFO 140461871777600] #progress_metric: host=algo-1, completed 80 % of epochs
#metrics {"Metrics": {"Max Batches Seen Between Resets": {"count": 1, "max": 400, 

[2020-08-16 06:24:14.589] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 100, "duration": 1008, "num_examples": 400, "num_bytes": 8461936}
[08/16/2020 06:24:14 INFO 140461871777600] #quality_metric: host=algo-1, epoch=49, train rmse <loss>=0.635659858794
[08/16/2020 06:24:14 INFO 140461871777600] #quality_metric: host=algo-1, epoch=49, train mse <loss>=0.404063456082
[08/16/2020 06:24:14 INFO 140461871777600] #quality_metric: host=algo-1, epoch=49, train absolute_loss <loss>=0.502538875341
[08/16/2020 06:24:14 INFO 140461871777600] #quality_metric: host=algo-1, train rmse <loss>=0.635659858794
[08/16/2020 06:24:14 INFO 140461871777600] #quality_metric: host=algo-1, train mse <loss>=0.404063456082
[08/16/2020 06:24:14 INFO 140461871777600] #quality_metric: host=algo-1, train absolute_loss <loss>=0.502538875341
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1011.1210346221924, "sum": 1011.1210346221924, "min": 1011.1210346221924}}, "EndTime

In [34]:
fm_predictor = fm.deploy(instance_type='ml.c4.xlarge', initial_instance_count=1)


Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


---------------!

In [35]:
sys.path.append('../')
import sagemaker_utils
from sagemaker_utils.query_serializer import serialize as fmserialize 
from sagemaker.predictor import  json_deserializer
from sklearn.metrics import accuracy_score
import numpy 


sagemaker_utils.query_serializer.nFeatures = nFeatures
fm_predictor.content_type = sagemaker_utils.query_serializer.CONTENT_TYPE
fm_predictor.serializer = fmserialize
fm_predictor.deserializer = json_deserializer


def model_accuracy(X_test, Y_test): 
    X_test_arr = X_test
        
    result = fm_predictor.predict(X_test_arr) 
    y_pred = [] 
    for p in result['predictions']: 
        if p['score'] > 0.5:
            y_pred.append(1)
        else: 
            y_pred.append(0)
    return accuracy_score(Y_test, y_pred, normalize=False)

In [36]:
accuracy = model_accuracy(X_test, Y_test) / len(Y_test)
accuracy 

0.5738735592036326

In [37]:
accuracy = model_accuracy(X_cold_test, Y_cold_test) / len(Y_cold_test)
accuracy

0.47191011235955055